In [1]:
import wmfdata as wmf
from wmfdata import charting, mariadb, hive
from wmfdata.utils import pct_str, pd_display_all
import requests
import re
from urllib import request
import json

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import jupyter_contrib_nbextensions
import datetime as dt 
from datetime import datetime, timedelta
import dateutil
import json

import logging
import gc
import weakref
import pprint

%load_ext sql_magic

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


## Functions
[Back to Table of Contents](#toc)

In [4]:
def merge_in(df, on="database_code"):
    global wikis
    wikis = pd.merge(wikis, df, how="left", on=on).fillna(0)

In [5]:
def get_iso_info_from_url():
    '''
    Grab ISO codes and return a list of country codes.
    '''
    iso_3166_url = "https://raw.githubusercontent.com/umpirsky/country-list/master/data/en/country.json"
    country_codes = json.loads(request.urlopen(iso_3166_url).read().decode())
    return country_codes

#canonical data in hive
#https://github.com/wikimedia-research/canonical-data/blob/master/countries.csv


## Filter definitions
[Back to Table of Contents](#toc)

In [2]:
#Parent class
class GlowProject:
    '''
    A Glow project.
    '''

    #Expect boolean
    #glow_project = glow_project
    
    _instances = set()
    
    def __init__(self, countries, wiki_codes, wiki_names):
        '''
        :param countries: names of the countries that are part of this project
        :type countries: list
        
        :param wiki_codes: abbreviated codes of the wikis that are part of this project
        :type wiki_codes: list
        
        :param wiki_names: full names of the wikis that are part of this project
                          (expected to be corresponding to the codes in `wiki_code`)
        :type wiki_names: list
        
        :param start_month: start month of the project (as a YYYY-MM formatted string)
        :type start_month: str
        
        :param end_month: end month of the project (as a YYYY-MM formatted string)
        :type end_month: str
        '''
        
        self.countries = countries
        self.wiki_codes = wiki_codes #wiki abbreviation
        self.wiki_names = wiki_names #language name for the wiki

        
        
        
    @classmethod
    def getinstances(cls):
        dead = set()
        for ref in cls._instances:
            obj = ref()
            if obj is not None:
                yield obj
            else:
                dead.add(ref)
        cls._instances -= dead
        
   
    def get_glow_meta_info(): #TO DO  
        for obj in GlowProject.get_instances():
            print (obj.countries)
    
    def get_countries(self):
        return (self.countries)
    
    def get_wiki_names(self):
        return(self.wiki_names)
    
    def get_wiki_codes(self):
        return(self.wiki_codes)
    
    def get_wiki_dbs_mariadb(self):
        '''
        Return a list of all the wiki_db+"wiki", databases,
        associated with this project for use with MariaDB queries.
        '''
        dbs = ["{}wiki".format(wiki_code.lower()) for wiki_code in self.wiki_codes]
        return(dbs)
    
    def get_quoted_wiki_dbs(self):
        '''
        Return a quoted, comma-separated list of the names of the wiki_dbs, databases,
        associated with this project.
        '''
        return(', '.join(['"{}wiki"'.format(wiki_code.lower()) for wiki_code in self.wiki_codes]))      

    def get_quoted_wiki_projects(self):
        '''
        Return a quoted, comma-separated list of the wiki projects 
        associated with this project. Example: "bn.wikipedia".
        '''
        return(', '.join(['"{}.wikipedia"'.format(wiki_code.lower()) for wiki_code in self.wiki_codes]))
    
    
    def get_iso_codes(self):
        '''
        Grab ISO codes and return a list of the codes for the defined countries of this project.
        '''
        country_codes = get_iso_info_from_url()
        country_codes = dict((v,k) for k,v in country_codes.items()) 
        
        iso_codes = list()
        for country in self.countries:
            try:
                iso_codes.append(country_codes[country])
            except KeyError:
                logging.warning('could not find the ISO code for {}'.format(country))
        return(iso_codes)    

    def get_quoted_iso_codes(self):
        '''
        Return a quoted, comma-separated list of the ISO codes for the countries
        associated with this project.
        '''
        return(', '.join(['"{}"'.format(iso_code) for iso_code in self.get_iso_codes()]))
    
    def get_quoted_glow_iso_codes(): #TO DO, remove the country hard coding 
        country_codes = get_iso_info_from_url()
        country_codes = dict((v,k) for k,v in country_codes.items()) #original country first, code 2nd
        glow_countries = ["India", "Argentina", "Nigeria", "Egypt", "Jordan", "Tunisia", "Algeria", "Morocco", "Lebanon", "Iraq", "Indonesia"]
        glow_iso_codes = [country_codes[x]for x in glow_countries]
        #return [country_codes[x]for x in GlowProject.get_quoted_glow_countries()]
        return(', '.join(['"{}"'.format(iso_code) for iso_code in glow_iso_codes]))
    
    def get_quoted_glow_countries():
        '''
        Return a quoted, comma-separated list of the country or countries 
        associated with the Glow project at large.
        '''
        glow_countries = []
        for obj in gc.get_objects():
            if isinstance(obj, GlowProject):
                glow_countries.append(obj.countries)
    
        glow_countries_list = [item for gatherlist in list(glow_countries) for item in gatherlist]
        return(', '.join(['"{}"'.format(country) for country in glow_countries_list]))
    
    def get_quoted_glow_wiki_codes():
        '''
        Return a quoted, comma-separated list of the wiki_codes 
        associated with the Glow project at large.
        '''
        glow_wiki_codes = []
        for obj in gc.get_objects():
            if isinstance(obj, GlowProject):
                glow_wiki_codes.append(obj.wiki_codes)
    
        glow_wiki_codes_list = [item for gatherlist in list(glow_wiki_codes) for item in gatherlist]
        return(', '.join(['"{}"'.format(wiki_code) for wiki_code in glow_wiki_codes_list]))
    
    def get_quoted_glow_wiki_dbs():
        '''
        Return a quoted, comma-separated list of the wiki_dbs 
        associated with the Glow project at large.
        '''
        glow_wiki_dbs = []
        for obj in gc.get_objects():
            if isinstance(obj, GlowProject):
                glow_wiki_dbs.append(obj.wiki_codes)
    
        glow_wiki_codes_dbs = [item for gatherlist in list(glow_wiki_dbs) for item in gatherlist]
        return(', '.join(['"{}wiki"'.format(wiki_code.lower()) for wiki_code in glow_wiki_codes_dbs]))
    
    def get_quoted_glow_wiki_projects():
        '''
        Return a quoted, comma-separated list of the wiki projects 
        associated with the Glow project at large. Example: "bn.wikipedia".
        '''
        glow_wiki_projects = []
        for obj in gc.get_objects():
            if isinstance(obj, GlowProject):
                glow_wiki_projects.append(obj.wiki_codes)
    
        glow_wiki_codes_projects = [item for gatherlist in list(glow_wiki_projects) for item in gatherlist]
        return(', '.join(['"{}.wikipedia"'.format(wiki_code.lower()) for wiki_code in glow_wiki_codes_projects]))
    

In [3]:
#start_month, start_year, end_month, end_year, articles, participants

glow_india = GlowProject(
    ['India'],
    ['BN','HI','ML','PA','TA','TE', 'AS','SA','KN','TCY','GU','BN','MR','SAT','UR', 'OR'],
    ['Bengali Wikipedia','Hindi Wikipedia','Malayalam Wikipedia','Punjabi','Tamil Wikipedia',
     'Telugu Wikipedia', 'Assamese Wikipedia', 'Sanskrit Wikipedia', 'Kannada Wikipedia', 
     'Tulu Wikipedia', 'Gujarati Wikipedia', 'Bengali Wikipedia', 'Marathi Wikipedia', 
     'Santali Wikipedia', 'Urdu Wikipedia', 'Odia Wikipedia'
    ],
    
    ##Adding start and end time of the project. In this
    ##case they're defined by today, TBD
    #(dt.datetime.today() - dt.timedelta(days=30)).strftime('%Y-%m'),
    #dt.datetime.today().strftime('%Y-%m'),
)


In [10]:
#fyi, big wikis list is hard coded... "wikidatawiki", "commonswiki","enwiki"

#list format query variables for querying databases that can be accessed via wmf.mariadb
india_glow_wiki_dbs_mariadb = glow_india.get_wiki_dbs_mariadb() 

#str/tuple/dt/datetime  format query variables for querying databases that can be accessed via wmf.hive 
query_vars = dict(
    india_wiki_dbs = glow_india.get_quoted_wiki_dbs(), 
    india_country_codes = glow_india.get_quoted_iso_codes(), 
    india_wiki_projects = glow_india.get_quoted_wiki_projects(),
    
    glow_country_codes = GlowProject.get_quoted_glow_countries(),
    glow_wiki_codes = GlowProject.get_quoted_glow_wiki_codes(),
    glow_wiki_dbs = GlowProject.get_quoted_glow_wiki_dbs(),
    glow_projects = GlowProject.get_quoted_glow_wiki_projects(),
    glow_iso_codes = GlowProject.get_quoted_glow_iso_codes(),
    

        
    #india_country = glow_india.get_quoted_countries(),
    MWH_SNAPSHOT = "2019-09",
    TODAY_DATE = dt.datetime.now().strftime('%Y-%m'),
    TODAY_DATE_FULL= dt.datetime.now().strftime('%Y-%m-%d'),
    TODAY_DATE_YEAR = dt.datetime.now().strftime('%Y'),
    TODAY_DATE_MONTH = dt.datetime.now().strftime('%m'),
    M_START_DATE = (datetime.today() - timedelta(days=30)).strftime('%Y-%m'),
    M_START_DATE_YEAR = (datetime.today() - timedelta(days=30)).strftime('%Y'),
    M_START_DATE_MONTH = (datetime.today() - timedelta(days=30)).strftime('%m'),  
    M2_START_DATE = (datetime.today() - timedelta(days=60)).strftime('%Y-%m'),
    Y_START_DATE = (datetime.today() - timedelta(days=365)).strftime('%Y-%m'),
    Y_START_DATE_FULL = (datetime.today() - timedelta(days=365)).strftime('%Y-%m-%d'),
    Y_START_DATE_YEAR = (datetime.today() - timedelta(days=365)).strftime('%Y'),
    Y_START_DATE_MONTH = (datetime.today() - timedelta(days=365)).strftime('%m'),
    Y2_START_DATE = (datetime.today() - timedelta(days=730)).strftime('%Y-%m'),
)

In [ ]:
#india country totals
pv_c_ts = hive.run("""
SELECT 
  country_code,
  SUM(view_count) as total,
  year, month
FROM wmf.pageview_hourly
  WHERE (year = '{M_START_DATE_YEAR}' AND month = '{M_START_DATE_MONTH}')
  AND agent_type='user'
  AND country_code = {india_country_codes}
  AND project IN ({india_wiki_projects})
GROUP BY 
  country_code, project, year, month
""".format(**query_vars))

In [348]:
india_total= pv_c_ts['total'].sum()
print(india_total)

95834182


In [353]:
#india country totals
pv_c_en = hive.run("""
SELECT 
  country_code,
  SUM(view_count) as total
FROM wmf.pageview_hourly
  WHERE (year = '{M_START_DATE_YEAR}' AND month = '{M_START_DATE_MONTH}')
  AND agent_type='user'
  AND country_code = {india_country_codes}
  AND project = 'en.wikipedia'
GROUP BY 
  country_code
""".format(**query_vars))

In [354]:
pv_c_en

,country_code,total
0,IN,661836299


In [342]:
#india project totals
pv_c_t = hive.run("""
SELECT 
  country_code,
  project,
  SUM(view_count) as total,
  year, month
FROM wmf.pageview_hourly
  WHERE (year = '{M_START_DATE_YEAR}' AND month = '{M_START_DATE_MONTH}')
  AND agent_type='user'
  AND country_code = {india_country_codes}
  AND project IN ({india_wiki_projects})
GROUP BY 
  country_code, project, year, month
""".format(**query_vars))

In [343]:
pv_c_t

,country_code,project,total,year,month
0,IN,sa.wikipedia,62586,2019,9
1,IN,tcy.wikipedia,38174,2019,9
2,IN,kn.wikipedia,2837748,2019,9
3,IN,sat.wikipedia,10617,2019,9
4,IN,ta.wikipedia,6830922,2019,9
5,IN,gu.wikipedia,1333832,2019,9
6,IN,pa.wikipedia,277253,2019,9
7,IN,bn.wikipedia,3833732,2019,9
8,IN,te.wikipedia,4210429,2019,9
9,IN,as.wikipedia,469649,2019,9


In [334]:
#https://github.com/wikimedia-research/Readers-core-metrics/blob/master/01_interactions_metrics.ipynb
#https://wikitech.wikimedia.org/wiki/Analytics/Data_Lake/Traffic/Pageview_hourly
pv_am_detail_r = hive.run("""
SELECT 
  country_code,
  project,
  page_id,
  referer_class,
  access_method,
  SUM(view_count) as am_total,
  SUM(IF(access_method = 'mobile app', view_count, null)) AS apps,
  SUM(IF(access_method = 'desktop', view_count, null)) AS desktop,
  SUM(IF(access_method = 'mobile web', view_count, null)) AS mobileweb,
  year, month
FROM wmf.pageview_hourly
  WHERE (year = '{M_START_DATE_YEAR}' AND month = '{M_START_DATE_MONTH}')
  AND agent_type='user'
  AND country_code = {india_country_codes}
  AND page_id != 0
  AND project IN ({india_wiki_projects})
GROUP BY 
  country_code, project, page_id, referer_class, access_method, year, month
""".format(**query_vars))

pv_am_detail = pv_am_detail_r.copy()
pv_am_detail.head()

,country_code,project,page_id,referer_class,access_method,am_total,apps,desktop,mobileweb,year,month
0,IN,as.wikipedia,4879,none,mobile web,48,NaN,NaN,48.0,2019,9
1,IN,as.wikipedia,5017,external (search engine),desktop,1,NaN,1.0,NaN,2019,9
2,IN,as.wikipedia,5689,internal,desktop,15,NaN,15.0,NaN,2019,9
3,IN,as.wikipedia,8095,unknown,mobile web,23,NaN,NaN,23.0,2019,9
4,IN,as.wikipedia,9304,unknown,mobile web,2,NaN,NaN,2.0,2019,9


In [18]:
pv_city_glow_r = hive.run("""
SELECT 
  country_code,
  project,
  city,
  year, 
  month
FROM wmf.pageview_hourly
  WHERE (year >= 2017)
  AND agent_type='user'
  AND country_code = 'IN'
  AND page_id != 0
  AND project IN ({india_wiki_projects})
GROUP BY 
  country_code, project, city, year, month
""".format(**query_vars))

In [28]:
pv_city_glow_r 

,country_code,project,city,year,month
0,IN,as.wikipedia,Bareilly,2017,8
1,IN,as.wikipedia,Chandannagar,2018,5
2,IN,as.wikipedia,Golaghat,2017,6
3,IN,as.wikipedia,Koch Bihar,2018,8
4,IN,as.wikipedia,Nallakunta,2018,10
...,...,...,...,...,...
391947,IN,ur.wikipedia,Nagapattinam,2018,2
391948,IN,ur.wikipedia,Pusad,2018,3
391949,IN,ur.wikipedia,Ranchi,2018,10
391950,IN,ur.wikipedia,Tiruvalla,2017,6


In [49]:
pv_city_r_en = hive.run("""
SELECT 
  country_code,
  project,
  city,
  year, 
  month
FROM wmf.pageview_hourly
  WHERE (year >= 2017)
  AND agent_type='user'
  AND country_code = 'IN'
  AND page_id != 0
  AND project ='en.wikipedia'
GROUP BY 
  country_code, project, city, year, month
""".format(**query_vars))

In [50]:
pv_city_r_en

,country_code,project,city,year,month
0,IN,en.wikipedia,Acharlagudem,2019,7
1,IN,en.wikipedia,Adraspalli,2018,1
2,IN,en.wikipedia,Alandur,2017,9
3,IN,en.wikipedia,Ambala,2017,2
4,IN,en.wikipedia,Apta,2017,1
...,...,...,...,...,...
95333,IN,en.wikipedia,Valsad,2017,2
95334,IN,en.wikipedia,Velim,2019,2
95335,IN,en.wikipedia,Wada,2018,3
95336,IN,en.wikipedia,Yamunanagar,2019,7


In [54]:
pv_glow_en_access_method_r = hive.run("""
SELECT 
  project,
  year, 
  access_method,
  SUM(view_count) as am_total
FROM wmf.pageview_hourly
  WHERE (year >= 2017)
  AND agent_type='user'
  AND country_code = 'IN'
  AND page_id != 0
  AND project IN ("bn.wikipedia", "hi.wikipedia", "ml.wikipedia", 
                  "pa.wikipedia", "ta.wikipedia", "te.wikipedia", "as.wikipedia", 
                  "sa.wikipedia", "kn.wikipedia", "tcy.wikipedia", "gu.wikipedia", 
                  "bn.wikipedia", "mr.wikipedia", "sat.wikipedia", "ur.wikipedia", 
                  "or.wikipedia", "en.wikipedia")
GROUP BY 
  project, year, access_method
""".format(**query_vars))

In [55]:
pv_glow_en_access_method_r

,project,year,access_method,am_total
0,kn.wikipedia,2018,mobile app,9452
1,mr.wikipedia,2018,desktop,7509554
2,en.wikipedia,2018,mobile web,5505874635
3,bn.wikipedia,2019,desktop,1905199
4,tcy.wikipedia,2019,mobile app,23
...,...,...,...,...
136,mr.wikipedia,2017,desktop,7055885
137,en.wikipedia,2017,mobile web,4342276550
138,pa.wikipedia,2019,desktop,283801
139,bn.wikipedia,2018,desktop,2290341


In [3]:
pv_glow_en_access_method_praveen = hive.run("""
SELECT 
  project,
  year, 
  access_method,
  SUM(view_count) as am_total
FROM wmf.pageview_hourly
  WHERE 
  year = 2019
  AND agent_type='user'
  AND country_code = 'IN'
  AND page_id != 0
  AND project = "mr.wikipedia"
GROUP BY project, year, access_method
""")

In [4]:
pv_glow_en_access_method_praveen

,project,year,access_method,am_total
0,mr.wikipedia,2019,desktop,5617379
1,mr.wikipedia,2019,mobile web,60511592
2,mr.wikipedia,2019,mobile app,14154


In [6]:
pv_glow_en_access_method_praveen2 = hive.run("""
SELECT 
  project,
  year, 
  access_method,
  SUM(view_count) as am_total
FROM wmf.pageview_hourly
  WHERE 
  year = 2019
  AND project = "mr.wikipedia"
GROUP BY project, year, access_method
""")

In [7]:
pv_glow_en_access_method_praveen2

,project,year,access_method,am_total
0,mr.wikipedia,2019,desktop,27013186
1,mr.wikipedia,2019,mobile web,75567749
2,mr.wikipedia,2019,mobile app,509134


### download data

In [52]:
pv_city_r_en.to_csv("./data/glow/india/pv_city_year_month_comparison_en.csv", sep=',', encoding = 'utf-8', index=False)

In [ ]:
pv_city_glow_r.to_csv("./data/glow/india/pv_city_year_month_comparison_glow.csv", sep=',', encoding = 'utf-8', index=False)

In [57]:
pv_glow_en_access_method_r.to_csv("./data/glow/india/pv_am_year_comparison_en_glow.csv", sep=',', encoding = 'utf-8', index=False)